In [153]:
! pip install -U scikit-learn
import pandas as pd
import datetime
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.inspection import plot_partial_dependence
import seaborn as sns
import pickle as pkl

Requirement already up-to-date: scikit-learn in /opt/anaconda3/lib/python3.7/site-packages (0.23.1)


In [172]:
sf = pd.read_csv('ca_san_francisco_2020_04_01.csv')

In [173]:
sf = sf.loc[sf.subject_race != 'other',:]

In [174]:
sf.columns

Index(['raw_row_number', 'date', 'time', 'location', 'lat', 'lng', 'district',
       'subject_age', 'subject_race', 'subject_sex', 'type', 'arrest_made',
       'citation_issued', 'warning_issued', 'outcome', 'contraband_found',
       'search_conducted', 'search_vehicle', 'search_basis', 'reason_for_stop',
       'raw_search_vehicle_description', 'raw_result_of_contact_description'],
      dtype='object')

In [175]:
# race_encoded = pd.get_dummies(sf.subject_race, drop_first=True)
le = LabelEncoder()
race_encoded = pd.DataFrame(le.fit_transform(sf.subject_race))
sex_dummy = pd.get_dummies(sf.subject_sex, drop_first=True)
sf = sf.drop(columns = ['raw_row_number','date','outcome','time','type','location','subject_race','subject_sex','lat', 'lng', 'district','citation_issued', 'warning_issued','reason_for_stop',
   'raw_search_vehicle_description', 'raw_result_of_contact_description','search_vehicle','search_basis'], axis=1)
sf.subject_age = sf.subject_age.dropna().apply(lambda x: str(int(x)) )
sf_sparse = pd.concat([sf,race_encoded,sex_dummy],axis=1)
sf_sparse.columns = ['subject_age','arrest_made','contraband_found','search_conducted','subject_race','gender']
sf_sparse = sf_sparse.dropna()

In [176]:
sf_sparse.head(5)

,subject_age,arrest_made,contraband_found,search_conducted,subject_race,gender
33182,39,True,False,True,0.0,1.0
33185,24,True,True,True,3.0,0.0
33192,24,False,False,True,3.0,1.0
33210,27,False,False,True,3.0,1.0
33211,48,False,False,True,0.0,1.0


In [ ]:
sf_sparse.to_pickle("./sf_transformed.pkl")
# sf_sparse = np.reshape(sf_sparse,(-1,1))
# scaler = StandardScaler()
# scaler.fit_transform(sf_sparse.subject_age)
# X = sf_sparse.dropna()
# sf.columns
clf = RandomForestClassifier(max_depth=2, random_state=46)
proto = clf.fit(X, y)
# X.columns

In [ ]:
proto.feature_importances_

In [ ]:
pd.DataFrame({'Variable':X.columns,
              'Importance':proto.feature_importances_}).sort_values('Importance', ascending=False)

In [ ]:
feat_importances = pd.Series(proto.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.xlabel("Feature Importances")
plt.ylabel("Variable")
plt.title("Feature Importances - Random Forest Classifier")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
gnb = GaussianNB()

In [ ]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [ ]:
nb = {
    'Class Count' : gnb.class_count_,
    'Class Prior' : gnb.class_prior_,
}

In [ ]:
nb

In [ ]:
train_score = gnb.score(X_train,y_train)

In [ ]:
test_score = gnb.score(X_test,y_test)

In [ ]:
train_score

In [ ]:
test_score

In [ ]:
features = [3,5]
pdp = plot_partial_dependence(proto, X, features)
plt.title('Partial Dependence of Arrest Outcome on\n' "Race with Random Forest")

In [ ]:
features = [6,4]
pdp = plot_partial_dependence(proto, X, features)

In [ ]:
X.subject_age = X.subject_age.astype('int')

In [ ]:
X = X.transpose()

In [ ]:
lr = LogisticRegression(penalty='l1',random_state=0,solver='liblinear',max_iter=1000).fit(X, y)

In [ ]:
lr.score(X,y)